In [10]:
import numpy as np
import time as tm
import datetime as dt
import tensorflow as tf

# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as pl

In [65]:
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv('NVDA_train.csv')
df.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'], inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df = df[['Date', 'Close']]
df.index = df.pop('Date')

# test_df = pd.read_csv('NVDA_test.csv')
# test_df.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'], inplace=True)
# test_df['Date'] = pd.to_datetime(test_df['Date'])
# test_df = test_df[['Date', 'Close']]
# test_df.index = test_df.pop('Date')


In [47]:
# plt.figure(figsize=(10, 6))
# plt.plot(train_df['Date'], train_df['Close'], marker='', linestyle='-')
# plt.title('Training Data: Close Values Over Time')
# plt.xlabel('Date')
# plt.ylabel('Close Value')
# plt.xticks(rotation=45)
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=(10, 6))
# plt.plot(test_df['Date'], test_df['Close'], marker='', linestyle='-')
# plt.title('Testing Data: Close Values Over Time')
# plt.xlabel('Date')
# plt.ylabel('Close Value')
# plt.xticks(rotation=45)
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [84]:
import datetime
import numpy as np

def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
  first_date = pd.to_datetime(first_date_str)
  last_date  = pd.to_datetime(last_date_str)

  target_date = first_date

  dates = []
  X, Y = [], []

  last_time = False
  while True:
    df_subset = dataframe.loc[:target_date].tail(n+1)

    if len(df_subset) != n+1:
      print(f'Error: Window of size {n} is too large for date {target_date}')
      return

    values = df_subset['Close'].to_numpy()
    x, y = values[:-1], values[-1]

    dates.append(target_date)
    X.append(x)
    Y.append(y)

    next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    year_month_day = next_date_str.split('-')
    year, month, day = year_month_day
    next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))

    if last_time:
      break

    target_date = next_date

    if target_date == last_date:
      last_time = True

  ret_df = pd.DataFrame({})
  ret_df['Target Date'] = dates

  X = np.array(X)
  for i in range(0, n):
    X[:, i]
    ret_df[f'Target-{n-i}'] = X[:, i]

  ret_df['Target'] = Y

  return ret_df

# Start day second time around: '2021-03-25'
windowed_df = df_to_windowed_df(df,
                                '2019-03-20',
                                '2023-12-29',
                                n=5)
windowed_df

,Target Date,Target-3,Target-2,Target-1,Target
0,2019-03-13,37.660000,40.285000,40.630001,42.154999
1,2019-03-14,40.285000,40.630001,42.154999,41.389999
2,2019-03-15,40.630001,42.154999,41.389999,42.452499
3,2019-03-18,42.154999,41.389999,42.452499,42.237499
4,2019-03-19,41.389999,42.452499,42.237499,43.927502
...,...,...,...,...,...
1205,2023-12-22,496.040009,481.109985,489.899994,488.299988
1206,2023-12-26,481.109985,489.899994,488.299988,492.790009
1207,2023-12-27,489.899994,488.299988,492.790009,494.170013
1208,2023-12-28,488.299988,492.790009,494.170013,495.220001
